<a href="https://colab.research.google.com/github/bhavithareddy-05/GenAi-vtu19980/blob/main/gpt_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00


In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset

In [3]:
# https://huggingface.co/datasets/hakurei/open-instruct-v1
dataset = load_dataset("hakurei/open-instruct-v1", split='train')
print(dataset.to_pandas().sample(5))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

       input                                        instruction  \
456452        you are a legislator. You are asked to come up...   
443227        how do I make a curl request that doesn't wait...   
411509        You are given a string and its length. your ta...   
481302                    Is yoga a good way to manage anxiety.   
280286        Re-write the following sentence to make it mor...   

                                                   output  
456452  As a legislator, I would propose the following...  
443227  You can make a `cURL` request that doesn't wai...  
411509                                          a,b,c,d,e  
481302  Yes, yoga is a great way to manage anxiety. It...  
280286  From designer stilettos to everyday sneakers, ...  


In [4]:
def preprocess(example):
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"
    return example

In [5]:
print(f"Before preprocessing: {dataset}")

Before preprocessing: Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 498813
})


In [6]:
dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])
print(f"After preprocessing: {dataset}")

Map:   0%|          | 0/498813 [00:00<?, ? examples/s]

After preprocessing: Dataset({
    features: ['prompt'],
    num_rows: 498813
})


In [7]:
dataset = dataset.shuffle(seed=42).select(range(100000)).train_test_split(test_size=0.1)
print(f"After train test split: {dataset}")

After train test split: DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['prompt'],
        num_rows: 10000
    })
})


In [8]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [9]:
# https://huggingface.co/microsoft/DialoGPT-medium
MODEL_NAME = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [10]:
print(f"First row Before tokenizing: {train_dataset['prompt'][0]}")

First row Before tokenizing: Provide an example of a positive and a negative impact of technology on society.  A positive impact of technology on society is increased access to information, which can help people gain new knowledge and skills. Additionally, advanced technology can improve efficiency and accuracy of processes, increasing productivity and making tasks easier and faster to complete. 

A negative impact of technology on society is data breaches, which can put individuals and companies at risk of identity theft and financial loss. Technologies can also be used to spy on citizens, which can lead to a loss of privacy and other civil liberties.


In [11]:
def tokenize_dataset(dataset):
    tokenized_dataset = dataset.map(lambda example:tokenizer(example['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])
    return tokenized_dataset

In [12]:
train_dataset = tokenize_dataset(train_dataset)
test_dataset = tokenize_dataset(test_dataset)
print(f"tokenized train dataset: {train_dataset}")
print(f"First row After tokenizing: {train_dataset['input_ids'][0], train_dataset['attention_mask'][0]}")

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

tokenized train dataset: Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 90000
})
First row After tokenizing: ([15946, 485, 281, 1672, 286, 257, 3967, 290, 257, 4633, 2928, 286, 3037, 319, 3592, 13, 220, 317, 3967, 2928, 286, 3037, 319, 3592, 318, 3220, 1895, 284, 1321, 11, 543, 460, 1037, 661, 4461, 649, 3725, 290, 4678, 13, 12032, 11, 6190, 3037, 460, 2987, 9332, 290, 9922, 286, 7767, 11, 3649, 13714, 290, 1642, 8861, 4577, 290, 5443, 284, 1844, 13, 220, 198, 198, 32, 4633, 2928, 286, 3037, 319, 3592, 318, 1366, 31523, 11, 543, 460, 1234, 3925, 290, 2706, 379, 2526, 286, 5369, 12402, 290, 3176, 2994, 13, 21852, 460, 635, 307, 973, 284, 13997, 319, 4290, 11, 543, 460, 1085, 284, 257, 2994, 286, 6782, 290, 584, 3026, 22008, 13], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='./dialogpt2-instruct',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

In [ ]:
print("training started...")
trainer.train()
print("training completed...")
trainer.save_model()
print("saved model...")

In [ ]:
# fine_tuned_model = AutoModelForCausalLM.from_pretrained('./dialogpt2-instruct').to('cuda')
fine_tuned_model = AutoModelForCausalLM.from_pretrained('TheFuzzyScientist/diabloGPT_open-instruct') #.to('cuda')

In [ ]:
def generate_text(prompt, model_selected):
    inputs = tokenizer.encode(prompt, return_tensors='pt') #.to('cuda')
    outputs = model_selected.generate(inputs, max_length=64, pad_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_token=True, use_mps_device=True)
    return generated_text[: generated_text.rfind('.')+1]

In [ ]:
print("Generating text from base model... ")
print(generate_text("I like to drink...", model))

Generating text from base model... 
I like to drink...


In [ ]:
print("Generating text from fine tuned model... ")
print(generate_text("I like to drink...", fine_tuned_model))

Generating text from fine tuned model... 
I like to drink...  I like to drink coffee.  I like to drink tea.  I like to drink coffee with milk.  I like to drink tea with milk.  I like to drink coffee with milk.  I like to drink tea with milk.  I like to drink coffee with milk.
